In [1]:
#extract neighGraph for concepts of clusters miltithreaded
import requests,os
import threading
import urllib
import time
import json
import importlib
import pickle
load = importlib.import_module("loadStuff")
cluster_neighbour = importlib.import_module("cluster_neighbour")
env = importlib.import_module("envf")
root_address = env.root_address
results = [[],[],[],[]]
counter=0
no_of_threads_already_running = 0
# Limit the number of threads.
pool = threading.BoundedSemaphore(50)




Not running on CoLab
Using mps device


/Users/baqia/miniforge3/envs/torch-gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def locally_fetch_dataset_concepts():
    filename = os.path.join(root_address +env.DATASET_NAME+ "/annotation.json")
    # Ensure the file exists before trying to open it
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            data = json.load(file)
            
            # Initialize a list to hold all the strings within the lists in all the values
            dataset_concepts = []
            
            # Iterate over the dictionary and extract all strings from the lists in the values
            for value in data.values():
                if isinstance(value, list):
                    for item in value:
                        if isinstance(item, str):
                            dataset_concepts.append(item)
    else:
        dataset_concepts = None

    return list(set(dataset_concepts))
def make_url_list(concepts):
    urls = []
    for concept in concepts:
        data = urllib.parse.urlencode([("lang", 'en'), ("title", concept),
            ("nPredLevels", 0), ("nSuccLevels", 1)])
        urls.append("http://www.wikifier.org/get-neigh-graph?" + data)
    return urls

def worker(u,concept_neighbour_local_dict,v):
    # Request passed URL.
    while True:
        try:
            r = requests.get(u)
            if r.status_code == 200:
                global counter
                counter+=1
                print("success"+ str(counter))
                concept_neighbour_local_dict[v] = r.text
                pool.release()
                return
            else:
                print("error")
                time.sleep(2)
        except:
            print("failure")



def req(concepts):
    concept_neighbour_local_dict = {}
    # Get URLs from a text file, remove white space.
    urls = make_url_list(concepts)
    no_of_threads_already_running = threading.active_count()
    for u,v in zip(urls,concepts):
        # Thread pool.
        # Blocks other threads (more than the set limit).
        pool.acquire(blocking=True)
        # Create a new thread.
        # Pass each URL (i.e. u parameter) to the worker function.
        t = threading.Thread(target=worker, args=(u,concept_neighbour_local_dict,v))
        # Start the newly create thread.
        t.start()

    #wait for all threads to finish
    while threading.active_count() > no_of_threads_already_running:
        print("waiting",threading.active_count())
        time.sleep(2)
        
    print("done")
    with open(root_address + env.DATASET_NAME +"/"+ "local_concept_neighbour.json", "w") as write_file:
        json.dump(concept_neighbour_local_dict, write_file)

def extract_neighbourhood_for_each_cluster():
    unique_concepts = load.unique_concepts_for_each_cluster()
    for i in range(len(unique_concepts)):
        try:
            req(i,unique_concepts[i])
        except:
            print("error in cluster ",i)

def extract_neighbourhood_for_dataset():
    dataset_concepts = locally_fetch_dataset_concepts()
    concept_neighbour_local_dict = req(dataset_concepts)
    


In [ ]:
extract_neighbourhood_for_dataset()

In [4]:
#load the local_concept_neighbour.json file



with open(root_address + env.DATASET_NAME +"/"+ "local_concept_neighbour.json", "r") as read_file:
    concept_neighbour_local_dict = json.load(read_file)

In [5]:
def extract_neighbourhood_for_each_cluster():
    unique_concepts = load.unique_concepts_for_each_cluster()
    with open(root_address + env.DATASET_NAME +"/"+ "local_concept_neighbour.json", "r") as read_file:
        concept_neighbour_local_dict = json.load(read_file)
    results = [[] for i in range(len(unique_concepts))]
    for i in range(len(unique_concepts)):
        for j in range(len(unique_concepts[i])):
            try:

                results[i].append({unique_concepts[i][j]:concept_neighbour_local_dict[unique_concepts[i][j]]})
            except:
                print("error in cluster ",i)
        with open(root_address + env.DATASET_NAME +"/"+ "cluster"+str(i)+"_neighbour.json", "w") as write_file:
            json.dump(results[i], write_file)

In [6]:
extract_neighbourhood_for_each_cluster()